# Challenge 2 - Colab Demo
This notebook is for Colab. The included OS commands are for linux and may not work on Windows machines.

## Setup Code

In [5]:
import torch
if torch.cuda.is_available():
  print("GPU used")
else:
  print("CPU used, slower execution")
  print("Try to get a GPU runtime for faster progress")

CPU used, slower execution
Try to get a GPU runtime for faster progress


In [3]:
!pwd
!ls

/Users/pulkit/Library/Mobile Documents/com~apple~CloudDocs/ai_fhtw/AI-ENG2024/ss2025/DevProjAIE/Challenge2/challenge2/DevProj-Challenge2
README.md                    models
challenge-2-colab-demo.ipynb requirements.txt
config.py                    test_crossval.py
dataset                      train_crossval.py


In [ ]:
# # activate to remove existing project to enable new git clone
# remove_project = True
# if remove_project:
#   # go to default directory - for Colab /content
#   %cd /content

#   !rm -rf Challenge2
# else:
#   !pwd
# !ls

In [ ]:
# !git clone https://github.com/Ma0256/Challenge2_2025.git Challenge2

In [ ]:
# %cd Challenge2
# !ls

In [ ]:
# installing all reqs can take a long time. First try to run with default package installation.
#!pip install -r requirements.txt

  Using cached llvmlite-0.44.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
Using cached llvmlite-0.44.0-cp312-cp312-macosx_11_0_arm64.whl (26.2 MB)


## Download Dataset

In [2]:
from dataset.dataset_ESC50 import ESC50
import config

testset = ESC50(subset="test", root=config.esc50_path, download=True)

data/esc50/master.zip: 616MiB [00:39, 16.4MiB/s] 


This dataset contains 2000 audio files and is pre-organized in 5 equal sized folds of 400 files. The predictive performance is measured via average accuracy of 5-fold cross validation.

What is the expected accuracy for random guessing? Check the number of classes.

In [3]:
config.n_classes

50

Look at the dataset here: https://github.com/karolpiczak/ESC-50

## Evaluate Cross Validation
This will test all 5 cross validation folds with training results of a sample-run - every fold with the respective model. First the sample-run (incl. the models) is downloaded. Then the model is loaded with the sample-run parameters and tested on the fold, for all 5 folds.

In [4]:
import pandas as pd
!mkdir results
!python test_crossval.py

download model params
results/sample-run.zip: 100%|███████████████| 499M/499M [00:09<00:00, 52.7MiB/s]
*****
AudioMLP(n_steps=431,n_mels=config.n_mels,hidden1_size=512,hidden2_size=128,output_size=config.n_classes,time_reduce=1)
*****

/Users/pulkit/Library/Mobile Documents/com~apple~CloudDocs/ai_fhtw/AI-ENG2024/ss2025/DevProjAIE/Challenge2/challenge2/DevProj-Challenge2/test_crossval.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlist

The mean CV accuracy should be TestAcc=0.361. This is quite good, given a random baseline of 0.02 / 2%.

The sample-run directory now contains two CSV files, one with test scores and another with predictions.

In [5]:
!ls results/sample-run/

1                        4                        test_scores_terminal.csv
2                        5
3                        test_probs_terminal.csv


Let's check the scores.

In [6]:
pd.read_csv('results/sample-run/test_scores_terminal.csv', index_col=0)

,TestAcc,TestLoss
1,0.362500,2.588383
2,0.330000,2.673482
3,0.347500,2.743354
4,0.412500,2.292052
5,0.352500,2.576224
mean,0.361000,2.574699
std,0.031105,0.171960


The predictions file is for submission. For every tested WAV file it contains the model predictions for the 50 classes.

In [7]:
pd.read_csv('results/sample-run/test_probs_terminal.csv', index_col=0).head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1-100032-A-0.wav,4.950584,4.086159,1.574152,-1.253273,-2.158790,5.638131,1.418871,-0.529921,-3.054562,0.565201,...,-1.460855,-3.771865,-2.689058,1.790378,-4.982672,-6.115510,-3.917398,-5.277284,2.484046,-0.621556
1-100038-A-14.wav,-1.823734,-0.081732,1.714846,-0.510304,-0.559352,-0.462725,-1.596903,-3.328725,3.402166,1.168820,...,-2.181345,-0.806688,2.740038,0.838290,-4.762272,-2.113644,1.220126,-1.318684,2.975007,3.918016
1-100210-A-36.wav,-6.004368,-4.552930,-0.429048,-1.860066,-2.542746,-2.629529,0.086764,0.783161,0.780406,1.507993,...,0.416693,5.807642,-1.925122,2.742118,5.625043,0.582538,-3.300045,2.434968,0.405135,3.650155
1-100210-B-36.wav,-6.082081,-4.593886,-0.688890,-2.084895,-2.785393,-2.881230,-0.118920,-0.366828,1.085200,1.281379,...,0.653679,5.467761,-2.095331,2.336657,5.946030,0.876772,-3.869635,2.541424,0.367543,3.366386
1-101296-A-19.wav,-2.992097,-7.215904,1.678491,0.729123,-3.915599,-0.457152,-1.054249,6.329288,-0.666291,-3.946541,...,4.721740,0.543420,-4.440724,1.899166,0.930295,5.349950,-2.209333,1.606775,3.384397,1.834558


Let's download the predictions file and submit it to the Challenge in Moodle.

## Train Cross Validation

In [8]:
!python train_crossval.py

*****
train folds are {2, 3, 4, 5} and test fold is {1}
random wave cropping

AudioMLP(n_steps=431,n_mels=config.n_mels,hidden1_size=512,hidden2_size=128,output_size=config.n_classes,time_reduce=1)
*****

 20%|████████▌                                  | 8/40 [00:11<00:35,  1.11s/bat]^C
  File "/Users/pulkit/miniconda3/envs/AI_FHTW/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/Users/pulkit/miniconda3/envs/AI_FHTW/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1327, in _next_data
    idx, data = self._get_data()
                ^^^^^^^^^^^^^^^^
  File "/Users/pulkit/miniconda3/envs/AI_FHTW/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1293, in _get_data
    success, data = self._try_get_data()
                    ^^^^^^^^^^^^^^^^^^^^
  File "/Users/pulkit/miniconda3/envs/AI_FHTW/lib/python3.12/site-packages/torch/utils/data/dataloader.py", lin